In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras import backend as K

class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = gamma

    def build(self, input_shape):
        # Create a trainable weight variable for the centers of the RBFs
        self.centers = self.add_weight(name='centers',
                                      shape=(self.units, input_shape[-1]),
                                      initializer='uniform',
                                      trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        # Calculate the radial basis functions for each input sample and each RBF center
        diff = K.expand_dims(inputs) - self.centers  # shape: (batch_size, units, input_dim)
        norm = K.sum(K.square(diff), axis=-1)  # shape: (batch_size, units)
        rbf = K.exp(-self.gamma * norm)  # shape: (batch_size, units)

        return rbf

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.units
    
url = 'https://raw.githubusercontent.com/ravsssh/HandballPrediction/main/Handball%20Prediction/datashotiqr/P.csv'
df = pd.read_csv(url, delimiter=',')
# Menentukan input(X) dan output (y)
X = df.drop(columns=["SHOT","GOAL","7MACC","6MACC","9MACC","WINGACC","PIVOTACC","FASTBREAKACC"])
y = df.GOAL
# Membagi dataset 80:20 untuk train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# normalisasi dataset agar tidak bias
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

validation_results_df = pd.DataFrame(columns=['R2_Test', 'MSE', 'MAE'])

for _ in range(20):
    model = Sequential()
    model.add(Flatten(input_shape=(6,)))
    model.add(RBFLayer(6, 0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='rmsprop', loss=mean_squared_error, metrics=['mean_absolute_error']) 
    history = model.fit(X_train, y_train, batch_size=8, epochs=100, validation_data=(X_test, y_test))
    
    pred_test = model.predict(X_test)
    
    r2_test = r2_score(y_test, pred_test)
    mse = history.history['val_loss'][-1]  # Last MSE from validation data
    mae = history.history['val_mean_absolute_error'][-1]  # Last MAE from validation data
    
    print(f"R-Squared Test Score: {r2_test}")
    
    # Append results to the validation results DataFrame
    validation_results_df = validation_results_df.append({'R2_Test': r2_test,
                                                          'MSE': mse,
                                                          'MAE': mae}, ignore_index=True)
    
    # Reset the TensorFlow session to clear the model
    tf.keras.backend.clear_session()

# Save the validation results DataFrame to a CSV file
validation_results_df.to_csv('validation_resultsshot.csv', index=False)


Epoch 1/100
300/300 [==============================] - 1s 2ms/step - loss: 3.5801 - mean_absolute_error: 1.3775 - val_loss: 2.5892 - val_mean_absolute_error: 1.3010
Epoch 2/100
300/300 [==============================] - 0s 1ms/step - loss: 2.4261 - mean_absolute_error: 1.2584 - val_loss: 2.0966 - val_mean_absolute_error: 1.1862
Epoch 3/100
300/300 [==============================] - 0s 1ms/step - loss: 2.0039 - mean_absolute_error: 1.1488 - val_loss: 1.8027 - val_mean_absolute_error: 1.0998
Epoch 4/100
300/300 [==============================] - 0s 1ms/step - loss: 1.7811 - mean_absolute_error: 1.0846 - val_loss: 1.6471 - val_mean_absolute_error: 1.0615
Epoch 5/100
300/300 [==============================] - 0s 1ms/step - loss: 1.6326 - mean_absolute_error: 1.0433 - val_loss: 1.5198 - val_mean_absolute_error: 1.0123
Epoch 6/100
300/300 [==============================] - 0s 1ms/step - loss: 1.4986 - mean_absolute_error: 0.9949 - val_loss: 1.3960 - val_mean_absolute_error: 0.9700
Epoch 7/10

C:\Users\Wandi\AppData\Local\Temp\ipykernel_150292\4012502756.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


300/300 [==============================] - 1s 2ms/step - loss: 3.9155 - mean_absolute_error: 1.4502 - val_loss: 2.2714 - val_mean_absolute_error: 1.1866
Epoch 2/100
300/300 [==============================] - 0s 1ms/step - loss: 2.2322 - mean_absolute_error: 1.2042 - val_loss: 2.0250 - val_mean_absolute_error: 1.1784
Epoch 3/100
300/300 [==============================] - 0s 1ms/step - loss: 2.0328 - mean_absolute_error: 1.1628 - val_loss: 1.8552 - val_mean_absolute_error: 1.1228
Epoch 4/100
300/300 [==============================] - 0s 1ms/step - loss: 1.8640 - mean_absolute_error: 1.1055 - val_loss: 1.6981 - val_mean_absolute_error: 1.0751
Epoch 5/100
300/300 [==============================] - 0s 1ms/step - loss: 1.7069 - mean_absolute_error: 1.0514 - val_loss: 1.5505 - val_mean_absolute_error: 1.0159
Epoch 6/100
300/300 [==============================] - 0s 1ms/step - loss: 1.5595 - mean_absolute_error: 0.9974 - val_loss: 1.4164 - val_mean_absolute_error: 0.9655
Epoch 7/100
300/300 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_150292\4012502756.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


300/300 [==============================] - 1s 1ms/step - loss: 4.0171 - mean_absolute_error: 1.4549 - val_loss: 2.5652 - val_mean_absolute_error: 1.2442
Epoch 2/100
300/300 [==============================] - 0s 1ms/step - loss: 2.4835 - mean_absolute_error: 1.2617 - val_loss: 2.2647 - val_mean_absolute_error: 1.2585
Epoch 3/100
300/300 [==============================] - 0s 1ms/step - loss: 2.2582 - mean_absolute_error: 1.2361 - val_loss: 2.0786 - val_mean_absolute_error: 1.1841
Epoch 4/100
300/300 [==============================] - 0s 1ms/step - loss: 2.0713 - mean_absolute_error: 1.1723 - val_loss: 1.9055 - val_mean_absolute_error: 1.1291
Epoch 5/100
300/300 [==============================] - 0s 1ms/step - loss: 1.8971 - mean_absolute_error: 1.1176 - val_loss: 1.7390 - val_mean_absolute_error: 1.0814
Epoch 6/100
300/300 [==============================] - 0s 1ms/step - loss: 1.7319 - mean_absolute_error: 1.0613 - val_loss: 1.5868 - val_mean_absolute_error: 1.0319
Epoch 7/100
300/300 [=

...

C:\Users\Wandi\AppData\Local\Temp\ipykernel_150292\4012502756.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


300/300 [==============================] - 1s 2ms/step - loss: 2.3807 - mean_absolute_error: 1.2096 - val_loss: 2.0695 - val_mean_absolute_error: 1.1940
Epoch 2/100
300/300 [==============================] - 0s 1ms/step - loss: 2.0717 - mean_absolute_error: 1.1710 - val_loss: 1.8754 - val_mean_absolute_error: 1.1326
Epoch 3/100
300/300 [==============================] - 0s 1ms/step - loss: 1.8719 - mean_absolute_error: 1.1068 - val_loss: 1.7029 - val_mean_absolute_error: 1.0721
Epoch 4/100
300/300 [==============================] - 0s 1ms/step - loss: 1.6951 - mean_absolute_error: 1.0477 - val_loss: 1.5445 - val_mean_absolute_error: 1.0177
Epoch 5/100
300/300 [==============================] - 0s 1ms/step - loss: 1.5322 - mean_absolute_error: 0.9901 - val_loss: 1.3953 - val_mean_absolute_error: 0.9544
Epoch 6/100
300/300 [==============================] - 0s 1ms/step - loss: 1.3897 - mean_absolute_error: 0.9364 - val_loss: 1.2678 - val_mean_absolute_error: 0.8990
Epoch 7/100
300/300 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_150292\4012502756.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


300/300 [==============================] - 1s 2ms/step - loss: 3.5676 - mean_absolute_error: 1.3653 - val_loss: 2.4825 - val_mean_absolute_error: 1.2579
Epoch 2/100
300/300 [==============================] - 0s 1ms/step - loss: 2.4464 - mean_absolute_error: 1.2727 - val_loss: 2.2313 - val_mean_absolute_error: 1.2136
Epoch 3/100
300/300 [==============================] - 0s 1ms/step - loss: 2.2175 - mean_absolute_error: 1.2083 - val_loss: 2.0231 - val_mean_absolute_error: 1.1763
Epoch 4/100
300/300 [==============================] - 0s 1ms/step - loss: 2.0119 - mean_absolute_error: 1.1541 - val_loss: 1.8486 - val_mean_absolute_error: 1.1066
Epoch 5/100
300/300 [==============================] - 0s 1ms/step - loss: 1.8379 - mean_absolute_error: 1.0909 - val_loss: 1.6854 - val_mean_absolute_error: 1.0701
Epoch 6/100
300/300 [==============================] - 0s 1ms/step - loss: 1.6688 - mean_absolute_error: 1.0410 - val_loss: 1.5306 - val_mean_absolute_error: 1.0057
Epoch 7/100
300/300 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_150292\4012502756.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,
